<a href="https://colab.research.google.com/github/zelatndnjs/Reverse_Dictionary/blob/main/Gemma_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터전처리

In [ ]:
import json
import os

In [ ]:
file_path = '.'
file_list = os.listdir(file_path)
json_list = [file for file in file_list if file.endswith('.json')]
print(json_list)

In [ ]:
def read_json(file_path, json_list):
  word_definition_list = []
  for i in range(len(json_list)):
    with open(os.path.join(file_path, json_list[i]), 'r',encoding='UTF-8') as f:
      json_data = json.load(f)
      channel = json_data['channel']
      total = channel['total']
      for ii in range(total):
        item = channel['item'][ii]

        wordinfo = item['wordinfo']
        word = wordinfo['word']

        senseinfo = item['senseinfo']
        definition = senseinfo['definition']
        if 'cat_info' in senseinfo :
          cat_info = senseinfo['cat_info']
          for iii in range(len(cat_info)):
            cat = cat_info[iii]['cat']
          word_definition_list.append([word, definition,cat])
        else:
          word_definition_list.append([word, definition])
  return word_definition_list
word_definition_list2 = read_json(file_path, json_list)
data_len=len(word_definition_list2)

In [ ]:
from collections import OrderedDict

json_data = OrderedDict()

json_data['data'] = []
for i in range(data_len):
    json_data['data'].append({"output" : word_definition_list2[i][0],"input":'' ,"instruction" : word_definition_list2[i][1]})

with open('output/data2.json','w',encoding = 'UTF-8') as f:
  json.dump(json_data, f, ensure_ascii=False, indent=4)

from collections import OrderedDict

json_data = OrderedDict()

json_data['data'] = []
for i in range(data_len):
    json_data['data'].append({"output" : word_definition_list2[i][0],"input":'' ,"instruction" : word_definition_list2[i][1]})

with open('output/data2.json','w',encoding = 'UTF-8') as f:
  json.dump(json_data, f, ensure_ascii=False, indent=4)

In [ ]:
data_path = 'drive/MyDrive/Colab Notebooks/data2.json'

In [ ]:
data = json.load(open(data_path,'r'))

In [ ]:
data['data'][0]

{'output': '게', 'input': '', 'instruction': '어떤 행동이 미치는 대상을 나타내는 격 조사.'}

In [ ]:
!pip install -U transformers
!pip install -U peft
!pip install -U trl
!pip install -U accelerate
!pip install -U bitsandbytes
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requ

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import transformers
import torch
import json

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset,Dataset, DatasetDict

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

BASE_MODEL = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
def generate_prompt(example):
  output_txt = []
  for i in range(len(example['instruction'])):
    message = [
        {"role":"user",
         "content":"{}라는 뜻을 가진 단어가 무엇인가요?\n".format(example['instruction'][i])},
        {"role":"assistant",
         "content": example['output'][i]}
    ]
    chat_message = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False)
    output_txt.append(chat_message)
  return output_txt

In [ ]:
import pandas as pd

train_dataset = Dataset.from_pandas(pd.DataFrame(data['data']))

dataset_dict = DatasetDict({
    "train": train_dataset
})

In [ ]:
train_data = dataset_dict['train']

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="output",
#        num_train_epochs = 1,
        max_steps=2700,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1181401 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
100,2.219700
200,1.620800
300,1.513800
400,1.542900
500,1.488800
600,1.506800
700,1.440100
800,1.493000
900,1.430100
1000,1.423300


TrainOutput(global_step=2700, training_loss=1.4702910811812788, metrics={'train_runtime': 4357.8327, 'train_samples_per_second': 2.478, 'train_steps_per_second': 0.62, 'total_flos': 8768243486048256.0, 'train_loss': 1.4702910811812788, 'epoch': 0.009141688554521285})

# 모델저장

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
!ls -alh lora_adapter

total 30M
drwxr-xr-x 2 root root 4.0K Sep 25 07:44 .
drwxr-xr-x 1 root root 4.0K Sep 25 07:44 ..
-rw-r--r-- 1 root root  721 Sep 25 07:44 adapter_config.json
-rw-r--r-- 1 root root  30M Sep 25 07:44 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Sep 25 07:44 README.md


In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = model.merge_and_unload()
#model.save_pretrained('gemma-2-2b-it-sum-ko')

In [ ]:
model.save_pretrained('drive/MyDrive/Colab Notebooks/gemma-2-2b-it-sum-ko')
#/content/drive/MyDrive/Colab Notebooks/gemma-2-2b-it-sum-ko

In [ ]:
!ls -alh ./gemma-2-2b-it-sum-ko

total 4.9G
drwxr-xr-x 2 root root 4.0K Sep 25 07:53 .
drwxr-xr-x 1 root root 4.0K Sep 25 07:49 ..
-rw-r--r-- 1 root root  880 Sep 25 07:49 config.json
-rw-r--r-- 1 root root  187 Sep 25 07:49 generation_config.json
-rw-r--r-- 1 root root 4.7G Sep 25 07:53 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root 230M Sep 25 07:53 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  24K Sep 25 07:53 model.safetensors.index.json


# Fine-tuned 모델 로드 및 추론

In [ ]:
BASE_MODEL = "google/gemma-2-2b-it"
FINETUNE_MODEL = "drive/MyDrive/Colab Notebooks/gemma-2-2b-it-sum-ko"

In [ ]:
del finetune_model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed Sep 25 08:54:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              28W /  70W |    103MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
import random
random_num = random.randrange(0,len(train_dataset['output']))
messages = [
    {
        "role":"user",
        "content":"다음 뜻을 가진 단어는 무엇인가요? : {}".format(train_dataset['instruction'][random_num])
    }
]
print(messages[0]['content'])

다음 뜻을 가진 단어는 무엇인가요? : 조선 시대에, 오위(五衛)에 속한 정오품 군직(軍職). 부호군 다음이며, 부사직의 위로 실무는 없었다.


In [ ]:
input_txt = "고양이의 방언"
messages = [
    {
        "role":"user",
        "content":"다음 뜻을 가진 단어는 무엇인가요? : {}".format(input_txt)
    }
]
print(messages[0]['content'])

다음 뜻을 가진 단어는 무엇인가요? : 고양이의 방언


In [ ]:
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample = True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

고양이-방


In [ ]:
print(messages[0]['content'])
print(outputs[0]["generated_text"][len(prompt):])

다음 뜻을 가진 단어는 무엇인가요? : 미국의 프레민저 감독이 만든 영화. 1959년에 제작됨
프레민저의 밤
